this notebook is the base of the code for the artwork [EIN STÜCK „LYRIK“: AUTOPOESIE APPARATUS FÜR ANTIKAPITALISTISCHE WERBEMITTEL](https://exmediawiki.khm.de/exmediawiki/images/3/35/KI-Plakat.pdf) from our former student [Verena Lercher](http://www.verenalercher.com/)  

---

# Text generation mit LSTM


## Implementierung der LSTM-Texterzeugung für Zeichen
![](./data/character-level_neural_language_model.png)

In [ ]:
import keras
keras.__version__

## Daten vorbereiten

laden und konvertieren in 'lowercase':

In [2]:
import keras
import numpy as np

#eigenes textfile (in UTF-8)
text = open("./data/KeinerWeissMehr.txt").read().lower()
print('Länge des Textkorpus:', len(text))


Länge des Textkorpus: 462081


umschreiben...:
1. sequenzen mit der länge `maxlen` extrahieren
2. mit `one-hot-encoding` in numpy-array `x` umwandeln
3. numpy-array mit der `Shape(sequences, maxlen, unique_characters)` speichern
4. numpy-array `y` für zielwerte vorbereiten

In [3]:
# Exraktion von 60-Zeichen-Sequenzen
maxlen = 60

# sampling einer neuen sequenz nach jeweils 3 zeichen
step = 3

# speichern der extrahierten sequenzen
sentences = []

# speichern der zielwerte (der vorhergesagten zeichen)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Anzahl der Sequenzen:', len(sentences))

# Liste der unterschiedlichen Zeichen erstellen
chars = sorted(list(set(text)))
print('Anzahl der in diesem Text verwendeten Zeichen:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# one-hot Kodierung in binäre Arrays
print('Vektorisierung...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Anzahl der Sequenzen: 154007
Anzahl der in diesem Text verwendeten Zeichen: 53
Vektorisierung...


## KNN erzeugen

1. ein einzelner `LSTM` layer 
2. ein `Dense` Klassifizierer
3. Berechnung der softmax-Funktion aller möglichen Zeichen

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

W1031 13:08:44.112322 140523881506624 deprecation_wrapper.py:119] From /home/student/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1031 13:08:44.123714 140523881506624 deprecation_wrapper.py:119] From /home/student/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1031 13:08:44.126005 140523881506624 deprecation_wrapper.py:119] From /home/student/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Da die Zielwerte one-hot encoded wurden, 
benutzen wir `categorical_crossentropy` als Verlustfunktion um das Netz zu trainieren:

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

W1031 13:08:44.406580 140523881506624 deprecation_wrapper.py:119] From /home/student/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1031 13:08:44.411983 140523881506624 deprecation_wrapper.py:119] From /home/student/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



## Trainieren des Sprachmodells und Sampling

Mit trainierten Modell und einem kleinen Anfangstext neue Texte erzeugen...:

1. Wahrscheinlichkeitsverteilung (anhand bereits erzeugten Textes) für das nächste Zeichen erzeugen
2. Neugewichtung für eine bestimmte Temperatur durchführen
3. das nächste Zeichen erzeugen
4. das neue Zeichen am Ende des Textes hizufügen

Code zur Neugewichtung und zum Abrufen eines Zeichenindex (Sampling-Funktion):

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finally - Das Modell trainieren und Text erzeugen

nach jeder Epoche verschieden Temperaturen verwenden

*...zum beobachten wie sich der erzeugte Text entwickelt und wie sich die verschiedenen Temperaturen der Sampling-Stratgie auswirken:*

In [7]:
import random
import sys

# Das Modell 60 Epochen lang trainieren
for epoch in range(1, 60):
    print('epoch', epoch)
    # Anpassung des Modells für einen Trainingsdurchlauf
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Zufällige Auswahl eines Anfangstexts
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Erzeuge Text mit dem Anfangswert: "' + generated_text + '"')
    
    # ausprobieren verschiedener Temperaturen
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ Temperatur:', temperature)
        sys.stdout.write(generated_text)

        # vom Anfangstext ausgehend 400 Zeichen erzeugen
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                # one-hot-codierung der schon erzeugten zeichen
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            # sampling des nächsten Zeichens
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

W1031 13:08:44.717007 140523881506624 deprecation.py:323] From /home/student/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


epoch 1


W1031 13:08:45.108004 140523881506624 deprecation_wrapper.py:119] From /home/student/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
154007/154007 [==============================] - 113s 731us/step - loss: 1.9120
--- Erzeuge Text mit dem Anfangswert: " saß,
wußte er, daß es gar nicht ihre starrheit war. es war "
------ Temperatur: 0.2
 saß,
wußte er, daß es gar nicht ihre starrheit war. es war nicht sein den stecke sich sehen sich seiner auch gesehen weiter weiter sich dann den schon den schon den scheiner stecken war nicht sein das zu sehen das war es weiter seine sich selber weiter war er war es das war er weiter sein er weiter dann war er weiter so sein man dann eine stecke sich das weiter das war so das war es das das wieder das dann das war in den schieben zu sehen war nicht das wa
------ Temperatur: 0.5
r das dann das war in den schieben zu sehen war nicht das war die war was weiter den straße zu sehen wegte, wenn es seiner so in den auch nicht den mehr konnte sie gesehen zusachen weiter setten einer gestracken wieder verstellen war, den konnte, das was und das weiter sehen war er das wester sein zu,

/home/student/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


hatte zu ange. nicht weiter woll, wenn es oder das sie hatte es einmal hinter des anderen stelle, andauernd war sie gehatten, des hellen kleid und wieder aus den stadt waren, weg. so foretzen könnte. wieder auf den
hatten, was er saß das räumch
------ Temperatur: 1.0
oretzen könnte. wieder auf den
hatten, was er saß das räumchungen, das mädchen und das
die
wußte er sehen, ihm bleibte, der, dabei nicht
die fremden. eine tazrezse unterwärm loufte, stadt zu wassen sie zählesehe bümlige halb und in richtig, was einer wollenssitzt verfällig, und des stand mußten, chläßten straße tropfte ständig von einem bistso waa
platz
des dehhes, de los reim gland in der hosehe verwisss, sie irgendom genammen noch auf bücherte und stehen
------ Temperatur: 1.2
verwisss, sie irgendom genammen noch auf bücherte und stehene prütset, dabei,
dehehe leute schlaff das kin diese waß g, die7eummolllich, 
losen woandert agserunelros, tristutung in derschöut.
da, ziehig mal. äußerst
michtig stadt war, es bok, aufha